In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn import metrics
import xgboost as xgb
import datetime

pd.set_option('display.float_format',lambda x: '%.5f'%x)


In [2]:
print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close()

f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)
dfPredAll = dfPredAll.set_index(dfPredAll['ID'])

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

Load data...
Done.


In [6]:
#number of NAN
feat_na = ['Total_NAN_Num']
dfTrain['Total_NAN_Num']=dfTrain[raw_predictors].T.isnull().sum().tolist()
dfPred['Total_NAN_Num']=dfPred[raw_predictors].T.isnull().sum().tolist()
for key,pro in procedure.items():
    tmp_pro = []
    for var in pro:
        if var in dfTrain.columns:
            tmp_pro.append(var)
    dfTrain[key+'_NAN_Num']=dfTrain[tmp_pro].T.isnull().sum().tolist()
    dfPred[key+'_NAN_Num']=dfPred[tmp_pro].T.isnull().sum().tolist()
    feat_na.append(key+'_NAN_Num')

In [7]:
#load features
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
feat_for_ensemble = [raw_predictors]
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/0_to_nan/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    feat_for_ensemble.append(tmp_feat_list)
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

In [ ]:
##define the variables
'''var_change = {'raw':raw}
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv',nrows=10)
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    var_change[filename] = {'constant':[],'category':[]}
    for var in tmp_feat_list:
        if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
            var_change[filename]['constant'].append(var)
        elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
            var_change[filename]['category'].append(var)
'''



In [8]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat+feat_na
for key,value in var_change.items():
    print(key)
    if key!='raw':
        continue
    for var in var_change[key]['constant']:
        if not var in predictors:
            continue
        predictors.remove(var)
    for var in var_change[key]['category']:
        if not var in predictors:
            continue
        predictors.remove(var)
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

raw
Feat_cnt_col
Feat_cnt_tool
Feat_pcent_col
Feat_pcent_tool


In [9]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5,param={'max_depth':4, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1.5,'subsample':0.8,
                         'colsample_bytree':0.8}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param  = param
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =50)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [11]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

[0]	train-rmse:2.33667	eval-rmse:2.31066
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42636	eval-rmse:1.40045
[100]	train-rmse:0.878988	eval-rmse:0.856285
[150]	train-rmse:0.551515	eval-rmse:0.53809
[200]	train-rmse:0.353	eval-rmse:0.354282
[250]	train-rmse:0.232232	eval-rmse:0.255504
[300]	train-rmse:0.158064	eval-rmse:0.206207
[350]	train-rmse:0.112462	eval-rmse:0.184387
[400]	train-rmse:0.084146	eval-rmse:0.17602
[450]	train-rmse:0.065925	eval-rmse:0.172613
[500]	train-rmse:0.054333	eval-rmse:0.171836
[550]	train-rmse:0.045453	eval-rmse:0.171942
Stopping. Best iteration:
[510]	train-rmse:0.052232	eval-rmse:0.171798

[0]	train-rmse:2.3271	eval-rmse:2.34962
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42043	eval-rmse:1.44248
[100]	train-rmse:0.876049	eval-r

In [ ]:
#ensemble experiment
input_round = 0
final_predictors = []
tool_dummies = []
for predictors in feat_for_ensemble:
    input_round+=1
    print(input_round)
    if input_round==1:
        for var in predictors:
            if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
                predictors.remove(var)
            elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
                predictors.remove(var)
                tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
                tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
                predictors = predictors + tmpTrain.columns.tolist()
                dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
                dfPred = pd.concat([dfPred,tmpPred],axis=1)
                if 'TOOL' in var:
                    tool_dummies += tmpTrain.columns.tolist()
    else:
        predictors+=tool_dummies
        '''for var in raw_predictors:
            if 'TOOL' in var:
                tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
                tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
                predictors = predictors + tmpTrain.columns.tolist()
                dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
                dfPred = pd.concat([dfPred,tmpPred],axis=1)'''             
    for var in predictors:
        if var not in dfPred.columns:
            dfPred[var] = 0                      
    ### model         
    tmp_test_result,tmp_result,imp = xgb_kfold(dfTrain,dfPred,predictors)
    
    tmp_result['score']=tmp_result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
    tmp_result = tmp_result[['ID','score']]
    
    tmp_test_result = tmp_test_result.rename(columns={'score':'score%d'%input_round})
    tmp_result = tmp_result.rename(columns={'score':'score%d'%input_round})
    
    final_predictors.append('score%d'%input_round)
    
    if input_round==1:
        final_test_result = tmp_test_result.copy(deep=True)
        final_result = tmp_result.copy(deep=True)
    else:
        final_test_result = final_test_result.merge(tmp_test_result[['ID','score%d'%input_round]],'inner','ID')
        final_result = final_result.merge(tmp_result[['ID','score%d'%input_round]],'inner','ID')
        
    
    
    

In [ ]:
final_test_result = final_test_result.merge(dfTrain[['ID','Y']],'inner','ID')

In [ ]:
test_result,result,imp = xgb_kfold(final_test_result,final_result,final_predictors,param={'max_depth':2, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1.5,'subsample':0.8,
                         'colsample_bytree':0.8})

In [12]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)

In [13]:
test_result.to_csv('../../Submission/test_version_%s.csv'%today,header=False,index=False)
imp.to_csv('../../Submission/imp_%s.csv'%today,header=False,index=False)